In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'/home/tari/Загрузки/taiga/KP'
filename = 'KP.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,KP,10@2644470,Навальному запретили выезжать из Кирова до кон...,www.kp.ru,Общество>Общество,,Екатерина БОЛГОВА,,,,2017-02-01,20:36:00+03:00,,http://www.kp.ru/online/news/2644470/
1,KP,10@2644469,Вопрос о застройке сквера на Нижнем озере в Ка...,www.kaliningrad.kp.ru,Общество>Общество,,Иван МАРКОВ,,,,2017-02-01,20:27:00+03:00,,http://www.kp.ru/online/news/2644469/
2,KP,10@2644468,Россия ввела пограничную зону на границе с Бел...,www.kp.ru,Политика>Политика,,Екатерина БОЛГОВА,,,,2017-02-01,20:26:00+03:00,,http://www.kp.ru/online/news/2644468/
3,KP,10@2644467,"В \""Новых Крестах\"" появятся три храма",www.spb.kp.ru,Общество>Общество,,Валерия ГАЛИУЛЛИНА,,,,2017-02-01,20:27:00+03:00,,http://www.kp.ru/online/news/2644467/
4,KP,10@2644466,В Волгограде экологи проведут две тысячи рейдов,www.volgograd.kp.ru,Общество>Общество,,Екатерина МАЛИНИНА,,,,2017-02-01,20:13:00+03:00,,http://www.kp.ru/online/news/2644466/


In [4]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [5]:
meta.iloc[6].to_dict()

{'author': 'ТКАЧ Лидия',
 'authorreaders': '',
 'authortexts': '',
 'date': '2017-02-01',
 'magazine': '',
 'segment': 'KP',
 'source': 'http://www.kp.ru/online/news/2644464/',
 'tags': '',
 'textdiff': '',
 'textid': '10@2644464',
 'textname': 'Молдавским работникам будут выдавать талоны на питание',
 'textregion': 'www.kp.md',
 'textrubric': 'Общество>Общество',
 'time': '20:17:00+03:00'}

In [ ]:
for i in tqdm(range(len(meta))):
    values = meta.iloc[i].to_dict()

    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()